---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.tail(10)

490                                  Lab: B12 969 2007\n
491                                   )and 8mo in 2009\n
492    .Moved to USA in 1986. Suffered from malnutrit...
493                                              r1978\n
494    . Went to Emerson, in Newfane Alaska. Started ...
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
dtype: object

In [3]:
def date_sorter():
    
    # Your code here
    
    import numpy as np
    
    in1_1 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    in1_2 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    in1 = pd.concat([in1_1,in1_2])
    in1.reset_index(inplace=True)
    in1_in = in1['level_0']

    in2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    in2.reset_index(inplace=True)
    in2_in = in2['level_0']

    in3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    in3.reset_index(inplace=True)
    in3_in = in3['level_0']

    in6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    in6.reset_index(inplace=True)
    in6_in = in6['level_0']
    
    store=[]
    for i in in6_in:
        if not(i in in1_in.values):
            store.append(i)
    store = np.asarray(store)
    in6 = in6[in6['level_0'].isin(store)]

    ## (7) 2009; 2010
    in7_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    in7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    in7 = pd.concat([in7_1,in7_2])
    in7.reset_index(inplace=True)

    in7_in = in7['level_0']
    
    store1=[]
    for i in in7_in:
        if not((i in in2_in.values) | (i in in3_in.values) | (i in in6_in.values)):
            store1.append(i)
    store1 = np.asarray(store1)
    in7 = in7[in7['level_0'].isin(store1)]
    
    all_index = in1.level_0.values.tolist()+in2.level_0.values.tolist()+in3.level_0.values.tolist()+in6.level_0.values.tolist()+in7.level_0.values.tolist()
    all_index = np.asarray(all_index)
    
    in1.columns=['level_0','match','month','day','year']
    in1['year']=in1['year'].apply(str)
    in1['year']=in1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    in2[1] = in2[1].apply(lambda x: x.replace(',',''))
    in2['day'] = in2[1].apply(lambda x:x.split(' ')[0])
    in2['year'] = in2[1].apply(lambda x:x.split(' ')[1])
    in2.columns=['level_0','match','month','day-year','day','year']
    in2.drop('day-year',axis=1,inplace=True) 
    
    in3.columns=['level_0','match','day','month','year']
    in3['day'] = in3['day'].replace(np.nan,-99)
    in3['day'] = in3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    in3['month'] = in3.month.apply(lambda x: x[:3])
    in3['month'] = pd.to_datetime(in3.month, format='%b').dt.month
    
    in6.columns=['level_0','match','month','year']
    in6['day']=1
  
    in7.columns=['level_0','match','year']
    in7['day']=1
    in7['month']=1
   
    f_df = pd.concat([in1,in2,in3,in6,in7])
    f_df['date'] =pd.to_datetime(f_df['month'].apply(str)+'/'+f_df['day'].apply(str)+'/'+f_df['year'].apply(str))
    f_df = f_df.sort_values(by='level_0').set_index('level_0')

    final_list = f_df['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(final_list), key=lambda x:x[1])],np.arange(500))

    return answer
    
    

In [4]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64